# SQL Language 

### Examples using Python 

as it works with most of SQL database	


In [6]:
#imports
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import sqlite3  

### SQL Queries

- Create/open a database

- Create a table in it

- Insert Data into table 

- Retrieve (View) table 

- Extract records that fulfill a specified criterion(a)

- Modify (Update) records

- Modify (Alter) columns


In [9]:
# STEP 1: Create a connection object that represents the database
conn = sqlite3.connect('MyDatabase.db')

print("Opened/Created database successfully")

Opened/Created database successfully


In [10]:
# STEP 2: Create a table in the database (using SQL CREATE TABLE statement)

conn.execute('''CREATE TABLE table1
         (ID INT PRIMARY KEY     NOT NULL,
         NAME           TEXT    NOT NULL,
         LOCATION       CHAR(50),
         EMAIL         CHAR(50));''')

print("Table created successfully")

Table created successfully


In [11]:
#STEP 3: Insert data into the table (using SQL INSERT INTO statement)

conn.execute("INSERT INTO table1 (ID,NAME,LOCATION,EMAIL) \
      VALUES (1, 'Mary', 'LA', 'm@123.com' )");

conn.execute("INSERT INTO table1 (ID,NAME,LOCATION,EMAIL) \
      VALUES (2, 'Alex', 'Miami', 'alex@123.com' )");

conn.execute("INSERT INTO table1 (ID,NAME,LOCATION,EMAIL) \
      VALUES (3, 'Kyra', 'NYC', 'kyra@123.com' )");

conn.execute("INSERT INTO table1 (ID,NAME,LOCATION,EMAIL) \
      VALUES (4, 'Nancy', 'SF', 'nancy@123.com' )");

conn.commit()
print("Records inserted successfully")

Records inserted successfully


In [12]:
# STEP 4: View table with Pandas (using SQL SELECT * FROM statement)

t1 = pd.read_sql("""SELECT * FROM table1;""", conn)
t1

,ID,NAME,LOCATION,EMAIL
0,1,Mary,LA,m@123.com
1,2,Alex,Miami,alex@123.com
2,3,Kyra,NYC,kyra@123.com
3,4,Nancy,SF,nancy@123.com


In [13]:
#SQL WHERE clause

#Example1: Get the NAME where LOCATION is NYC:

pd.read_sql("""SELECT NAME FROM table1
                    WHERE LOCATION = 'NYC';""", conn)

,NAME
0,Kyra


In [14]:
#Example2: Get the NAME AND EMAIL where LOCATION is SF or NYC:

pd.read_sql("""SELECT NAME, EMAIL FROM table1
                            WHERE LOCATION = 'NYC' OR LOCATION ='SF';""", conn)

,NAME,EMAIL
0,Kyra,kyra@123.com
1,Nancy,nancy@123.com


In [15]:
#SQL UPDATE Query is used to modify row(s) in table (use WHERE clause to specifies which record(s) should be updated)

#Example: To change the lOCATION for Alex from Miami to Montreal use update query:

query = """UPDATE table1 SET LOCATION = 'Montreal' WHERE NAME = 'Alex';"""

conn.execute(query)#execute the query
conn.commit()

In [16]:
#Check if the Alex record was updated: 
pd.read_sql("""SELECT LOCATION FROM table1
                            WHERE NAME = 'Alex';""", conn)

,LOCATION
0,Montreal


In [17]:
#View updated table

pd.read_sql("""SELECT * FROM table1;""", conn)

,ID,NAME,LOCATION,EMAIL
0,1,Mary,LA,m@123.com
1,2,Alex,Montreal,alex@123.com
2,3,Kyra,NYC,kyra@123.com
3,4,Nancy,SF,nancy@123.com


In [18]:
# SQL DELETE Query to delete row(s) from a table (use WHERE clause to specifies which record(s) should be deleted)

#Example: To delete the last row use the following delete query:
query = """DELETE FROM table1 WHERE ID = 4;"""

#execute the query
conn.execute(query)
conn.commit()

In [19]:
#View table to check if 4th row was deleted

pd.read_sql("""SELECT * FROM table1;""", conn)

,ID,NAME,LOCATION,EMAIL
0,1,Mary,LA,m@123.com
1,2,Alex,Montreal,alex@123.com
2,3,Kyra,NYC,kyra@123.com


In [20]:
#SQL ALTER TABLE to add/delete/rename columns or change columns constraints (changes the structure of table)


#Example1: Add a column age of integer type to the table using the following ALTER TABLE query:

query = """ALTER TABLE table1 ADD age int;"""

#execute the query
conn.execute(query)
conn.commit()

In [21]:
#View table to check if new column was added

pd.read_sql("""SELECT * FROM table1;""", conn)

,ID,NAME,LOCATION,EMAIL,age
0,1,Mary,LA,m@123.com,None
1,2,Alex,Montreal,alex@123.com,None
2,3,Kyra,NYC,kyra@123.com,None


In [22]:
#Insert data for age into the table using the following SQL UPDATE Queries:

conn.execute("UPDATE table1 SET age = 33 where Name = 'Mary'");

conn.execute("UPDATE table1 SET age = 38 where Name = 'Alex'");

conn.execute("UPDATE table1 SET age = 39 where Name = 'Kyra'");

In [23]:
#View table to check if new column was added

pd.read_sql("""SELECT * FROM table1;""", conn)

,ID,NAME,LOCATION,EMAIL,age
0,1,Mary,LA,m@123.com,33
1,2,Alex,Montreal,alex@123.com,38
2,3,Kyra,NYC,kyra@123.com,39


We used SQL to query a table. 

In practice, data is shared accross multiple tables. Below queries multiple tables using SQL Joins:

### SQL Joins: 


- inner join

- left join

- right join

- full outer join

- cross join



In [24]:
#Create a new table in our database (using SQL CREATE TABLE statement)

conn.execute('''CREATE TABLE table2
         (ID INT PRIMARY KEY     NOT NULL,
         SALARY REAL);''')

print("Table created successfully")

Table created successfully


In [25]:
#Insert data into the table (using SQL INSERT INTO statement)

conn.execute("INSERT INTO table2 values (1, 55000)");

conn.execute("INSERT INTO table2 values (2, 70000)");

conn.execute("INSERT INTO table2 values (3, 100000)");

In [26]:
#View table to check if new column was added

t2= pd.read_sql("""SELECT * FROM table2;""", conn)
t2

,ID,SALARY
0,1,55000.0
1,2,70000.0
2,3,100000.0


In [27]:
#Deleting row in table1 using SQL DELETE Query

query = """DELETE FROM table1 WHERE ID = 3;"""

#execute the query
conn.execute(query)
conn.commit()

In [28]:
#View table to check if 4th row was deleted

pd.read_sql("""SELECT * FROM table1;""", conn)

,ID,NAME,LOCATION,EMAIL,age
0,1,Mary,LA,m@123.com,33
1,2,Alex,Montreal,alex@123.com,38


In [29]:
#SQL INNER JOIN will give the common columns from both tables

conn.execute("SELECT A.NAME, A.LOCATION, B.SALARY FROM table1 A INNER JOIN table2 B ON A.ID = B.ID").fetchall()

[('Mary', 'LA', 55000.0), ('Alex', 'Montreal', 70000.0)]

In [30]:
#SQL LEFT JOIN will give ALL columns from table1 and common columns

conn.execute("SELECT A.NAME, A.LOCATION, B.SALARY FROM table1 A LEFT JOIN table2 B ON A.ID = B.ID").fetchall()

[('Mary', 'LA', 55000.0), ('Alex', 'Montreal', 70000.0)]

In [31]:
Although Python Jupiter supports only left and inner joins, other joins queries may be useful:

#SQL RIGHT JOIN will give ALL columns from table2 and common columns

query ="SELECT A.NAME, A.LOCATION, B.SALARY FROM table1 A RIGHT JOIN table2 B ON A.ID = B.ID";

In [32]:
#SQL FULL OUTER JOIN will give common columns and columns from table1 & table2 where missing values are replaced with None

query = "SELECT A.NAME, A.LOCATION, B.SALARY FROM table1 A CROSS JOIN table2 B ON A.ID = B.ID";


In [33]:
#SQL CROSS JOIN will give all rows from table1 combined with all rows from table2 

query = "SELECT A.NAME, A.LOCATION, B.SALARY FROM table1 A CROSS JOIN table2 B ON A.ID = B.ID";



### SQL Aggregation & Date Functions

In [36]:
pd.read_sql("""SELECT * FROM table2;""", conn)

,ID,SALARY
0,1,55000.0
1,2,70000.0
2,3,100000.0


In [40]:
#Average
pd.read_sql("""SELECT AVG(SALARY) from table2;""", conn)

,AVG(SALARY)
0,75000.0


In [41]:
#count
pd.read_sql("SELECT COUNT(SALARY) FROM table2;", conn)

,COUNT(SALARY)
0,3


In [42]:
#sum
pd.read_sql("SELECT SUM(SALARY) FROM table2;", conn)

,SUM(SALARY)
0,225000.0


In [43]:
#MAX/MIN
pd.read_sql("SELECT MAX(SALARY) FROM table2;", conn)

,MAX(SALARY)
0,100000.0


In [58]:
#DATE 
pd.read_sql("SELECT DATE();", conn)

,DATE()
0,2021-04-23
